# DataKind DC - Red Cross Phase 2
## Smoke Alarm Model 1b: Code reproduction

Maria Brun <br>
April, 2018

In [8]:
import pandas as pd
import numpy as np
import scipy
from sklearn import linear_model

#This is old and causes warnings - updated with sklearn.model_selection
# from sklearn.cross_validation import train_test_split, cross_val_score

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import normalize, Imputer
from sklearn.metrics import mean_squared_error
from math import sqrt

#Get the working directory

import os
os.getcwd()

#Change the working directory and confirm it changed to where you want it to be!

os.chdir('C:\\Users\\mcboe72\\RedCross')
os.getcwd()

'C:\\Users\\mcboe72\\RedCross'

In [12]:
tract_data = pd.read_csv('2009_2013_alarm_presence_by_tract.csv')
del tract_data['Unnamed: 0']
tract_data.head()

,tractid,all_fire_all_years,alarm_unknown_all_years,alarm_presented_all_years,alarm_not_presented_all_years,ratio_no_alarm_in_all,ratio_no_alarm_in_all_known
0,1001020100,4,0,4,0,0.000000,0.000000
1,1001020200,8,1,3,4,0.500000,0.571429
2,1001020300,7,2,0,5,0.714286,1.000000
3,1001020400,8,4,2,2,0.250000,0.500000
4,1001020500,15,5,7,3,0.200000,0.300000


In [10]:
acs = pd.read_csv('http://enigma-public.s3.amazonaws.com/projects/smoke-alarm-risk/data/acs.csv')

In [61]:
acs.head()

,geoid,tenure_renter_occupied,tenure_owner_occupied,vacancy_for_seasonal_recreational,vacancy_for_rent,vacancy_for_sale_only,vacancy_other_vacant,vacancy_rented_not_occupied,vacancy_for_migrant_workers,vacancy_sold_not_occupied,...,hhmove_moved_in_2010_or_later,qfs1_yes,qfs1_no,hdsb_no,hdsb_yes,geoid_parsed,sum_level,state,cnty,raw_tract
0,07000US021986431016360,0.202703,0.797297,0.712871,0.148515,0.000000,0.138614,0.0,0.0,0.000000,...,0.202703,0.108108,0.891892,0.000000,0.033613,021986431016360,07000,02,198,6431016360
1,15000US020900019004,0.142431,0.857569,0.557778,0.126667,0.000000,0.315556,0.0,0.0,0.000000,...,0.155542,NaN,NaN,NaN,NaN,020900019004,15000,02,090,0019004
2,15000US021220005002,0.206806,0.793194,0.912621,0.000000,0.048544,0.038835,0.0,0.0,0.000000,...,0.117801,NaN,NaN,NaN,NaN,021220005002,15000,02,122,0005002
3,26900US63150231050,0.371134,0.628866,0.027027,0.081081,0.000000,0.891892,0.0,0.0,0.000000,...,0.231959,0.216495,0.783505,0.015385,0.010256,63150231050,26900,63,150,231050
4,26900US64800222910,0.286408,0.713592,0.000000,0.038462,0.000000,0.807692,0.0,0.0,0.153846,...,0.092233,0.359223,0.640777,0.002370,0.008294,64800222910,26900,64,800,222910


The geoid is a code that indicates the summary level, the state, county, etc <br>
For reference, see the documentation at this website: https://www.census.gov/geo/reference/geoidentifiers.html <br> <br>
The number of digits determins the level: 2 for state, 5 for state and county, 10 for state and county and county subdivision, etc <br> <br>
This code splits the column geoid into tract ids - parsed ID and sum level <br>
As an example, I output the full geoid and the parsed ID and sum level for the 100th observation in a list

In [68]:
acs['geoid_parsed'] = [geo[1] for geo in acs['geoid'].str.split('US')]
acs['sum_level'] = [geo[0] for geo in acs['geoid'].str.split('US')]

examples_geoid = (acs['geoid'][100], acs['geoid_parsed'][100], acs['sum_level'][100])
examples_geoid

('15000US060374607001', '060374607001', '15000')

In [69]:
#Creates new variables state, county, etc, by pulling out the part of the geoid that corresponds to each 
#Commented out of code

#acs['state'] = acs['geoid_parsed'].str[0:2]
#acs['cnty'] = acs['geoid_parsed'].str[2:5]
#acs['raw_tract'] = acs['geoid_parsed'].str[5:]

In [70]:
#Create a new data set with just the census tract level summary data

acsCT = acs.query("sum_level == '14000'")
acsCT.head()

,geoid,tenure_renter_occupied,tenure_owner_occupied,vacancy_for_seasonal_recreational,vacancy_for_rent,vacancy_for_sale_only,vacancy_other_vacant,vacancy_rented_not_occupied,vacancy_for_migrant_workers,vacancy_sold_not_occupied,...,hhmove_moved_in_2010_or_later,qfs1_yes,qfs1_no,hdsb_no,hdsb_yes,geoid_parsed,sum_level,state,cnty,raw_tract
32,14000US05015950500,0.356879,0.643121,0.219365,0.124054,0.128593,0.390318,0.027231,0.0,0.110439,...,0.205749,0.198357,0.801643,0.019764,0.034319,05015950500,14000,05,015,950500
33,14000US05033020302,0.273465,0.726535,0.259740,0.415584,0.000000,0.324675,0.000000,0.0,0.000000,...,0.201043,0.092700,0.907300,0.014880,0.016411,05033020302,14000,05,033,020302
34,14000US05045030702,0.619534,0.380466,0.000000,0.469388,0.178571,0.112245,0.239796,0.0,0.000000,...,0.255102,0.203353,0.796647,0.018452,0.023134,05045030702,14000,05,045,030702
35,14000US05057480500,0.503280,0.496720,0.000000,0.222222,0.057143,0.644444,0.076190,0.0,0.000000,...,0.162791,0.191413,0.808587,0.021086,0.032425,05057480500,14000,05,057,480500
36,14000US05085020103,0.089406,0.910594,0.212245,0.159184,0.289796,0.220408,0.000000,0.0,0.118367,...,0.121447,0.067183,0.932817,0.008466,0.016740,05085020103,14000,05,085,020103


In [71]:
acsCT = acsCT.rename(columns = {'geoid_parsed':'tractid'})

#This code errors 
#acsCT['tractid'] = acsCT['tractid'].astype(int)

#Instead, it is possible to use float, but this seems like an unnecessary step
#acsCT['tractid'] = acsCT['tractid'].astype(float)

acs_tractid = acsCT['tractid']
acsCT = acsCT.drop(['sum_level', 'geoid'], axis = 1)

Thsi code imputes the mean for missing values

In [72]:
acs_features = acsCT.drop(['tractid'], axis = 1)
col_names = acs_features.columns

In [73]:
imp = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0).fit(acs_features)

#Renamed dataframe to avoid recursively referencing a dataframe

acs_features_imp = imp.transform(acs_features)

acsCT_imp = pd.DataFrame(acs_features_imp)
acsCT_imp.columns = col_names
acsCT = pd.concat([acsCT.reset_index(drop=True), acs_tractid.reset_index(drop=True)], axis = 1)

Merge alarm presence by tract data with the census tract level ACS data

In [74]:
tract_data_merged = tract_data.merge(acsCT, how='left', on='tractid')

ValueError: Buffer has wrong number of dimensions (expected 1, got 2)